Dependencies | Version
--- | ---
SatPy | 0.9.0
PyResample | 1.10.1
Trollimage | 1.5.3
PyKdtree | 1.3.1


# Quickstart with EARS-NWC

<p>In December 2012 <a class="reference external" href="http://www.eumetsat.int/">EUMETSAT</a> extended the <a class="reference external" href="http://www.eumetsat.int/home/main/satellites/groundnetwork/earssystem/index.htm">EARS</a> services to include a new service with the purpose of supporting European users with cloud information
from polar orbiting satellites in near real time.</p>
<p>This <em>EARS-NWC</em> service provides the parameters <em>Cloud Mask</em>, <em>Cloud Type</em> and
<em>Cloud Top Temperature and Height (CTTH)</em> as derived using the Polar Platform
System (<a class="reference external" href="http://nwcsaf.smhi.se/">PPS</a>) software package from the <a class="reference external" href="http://www.nwcsaf.org/">NWC SAF</a>. The products are derived
from AVHRR data received at the EARS core stations with a reception coverage including Europe and the North Atlantic. Products are disseminated on <a class="reference external" href="http://www.eumetsat.int/home/main/dataaccess/eumetcast/index.htm">EUMETCast</a> (EUMETSAT <a class="reference external" href="http://www.eumetsat.int/home/main/dataaccess/eumetcast/receptionstationset-up/sp_20100623124251305?l=en">data channel 1</a>) with a timeliness better than 30
minutes, and available in netCDF4 format. The geolocation information is available on a tie-point grid and stored in each product.
<p>At the moment the satellites contributing to the service are Metop-B and NOAA-19.</p>

## Loading and concatenating the cloud type granules

In [1]:
from satpy import Scene
from satpy import find_files_and_readers
from datetime import datetime

DATA_DIR = "/home/a000680/laptop/Nordisk/EARS_NWC/data/case_20180314/ears"

myfiles = find_files_and_readers(base_dir=DATA_DIR,
                                 start_time=datetime(2018, 3, 14, 10, 5),
                                 end_time=datetime(2018, 3, 14, 10, 11),
                                 reader='nc_nwcsaf_pps')
scene = Scene(filenames=myfiles)
scene.load(['ct'])

(0, ['/home/a000680/usr/src/satpy/satpy/etc/readers/nc_nwcsaf_pps.yaml'], ['nc_nwcsaf_pps'])


/usr/lib64/python2.7/_abcoll.py:391: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  return list(self)
/usr/lib64/python2.7/_abcoll.py:391: FutureWarning: calling len() on an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Call len() on the Dataset.variables property instead, like ``len(ds.variables)``, to preserve existing behavior in a forwards compatible manner.
  return list(self)
/home/a000680/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Now we have loaded and concatenated the <em>ct</em> field of the Cloudtype product granules in the time window given by the start amd end times above

In [2]:
print(scene['ct'].data.shape)
print(scene['ct'].data.compute())

(4320, 2048)
[[ 2  2  2 ...  6  6  6]
 [ 2  2  2 ...  6  6  6]
 [ 2  2  2 ... 10  6  6]
 ...
 [ 7  6  6 ...  2  2  2]
 [ 7  6  6 ...  2  2  2]
 [ 7  6  6 ...  2  2  2]]


<p>Also the geolocation has been unpacked. That is the the full resolution
geolocation information has been recreated from the tie point grid by
interpolating and extrapolating the longitudes and latitudes on the tie point
grid. This is accomplished using the <a class="reference external" href="http://www.github.com/adybbroe/python-geotiepoints">python-geotiepoints</a> tool, but this is
transparent to the user:</p>

In [3]:
print(scene['ct'].area.lats.shape)

(4320, 2048)


Now let’s visualise the cloudtype data using the Nowcasting SAF palette read from the file:

In [4]:
scene.load(['cloudtype'])
scene.show('cloudtype')

<img src='https://github.com/pytroll/pytroll-examples/raw/master/satpy/ears_nwc_cloudtype_20180314T1005Z_20180314T1010Z_thumb.png'>

# Load all CTTH granules, concatenate, reproject, generate composite image, and save to disk

In [5]:
from glob import glob
import os.path

In [6]:
DATA_DIR = "/home/a000680/laptop/Nordisk/EARS_NWC/data/case_20180314/ears"

## Make a list of all CTTH files in the directory, both bzipped and unpacked ones:

In [7]:
myfiles = glob(os.path.join(DATA_DIR, "*CTTH_C*.nc.bz2"))
myfiles = myfiles + glob(os.path.join(DATA_DIR, "*CTTH_C*.nc"))

## Generate the scene object

In [8]:
scene = Scene(filenames=myfiles, reader='nc_nwcsaf_pps')

## Load the cloud height composite image

In [9]:
scene.load(['cloud_top_height'])

/home/a000680/.local/lib/python2.7/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in less_equal
  return func(*args2)
/home/a000680/.local/lib/python2.7/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)


## Resample the data to the pre-defined "euron1" area

In [10]:
lscn = scene.resample('euron1', radius_of_influence=5000)

No handlers could be found for logger "pyresample.kd_tree"


In [11]:
lscn.show('cloud_top_height')

<img src='https://github.com/pytroll/pytroll-examples/raw/master/satpy/ears_nwc_ctth_20180314T1000Z_20180314T1014Z_thumbnail.png'>

## Save the image to file

In [11]:
dt_start = lscn.attrs['start_time'].strftime('%Y%m%dT%H%MZ')
dt_end = lscn.attrs['end_time'].strftime('%Y%m%dT%H%MZ')
lscn.save_dataset(
    'cloud_top_height', filename='./ears_nwc_ctth_{starttime}_{endtime}.png'.format(starttime=dt_start, endtime=dt_end))